In [ ]:
from IPython.core.display import HTML

#with open('creative_commons.txt', 'r') as f:
#    html = f.read()
    
name = '2014-03-10-google-earth'

#html = """
#<small>
#<p> This post was written as an IPython notebook.  It is available for
#<a href="https://ocefpaf.github.com/python4oceanographers/downloads/
#notebooks/%s.ipynb">download</a> or as a static
#<a href="https://nbviewer.ipython.org/url/ocefpaf.github.com/
##python4oceanographers/downloads/notebooks/%s.ipynb">html</a>.</p>
#<p></p>
#%s """ % (name, name, html)

%matplotlib inline
from matplotlib import style
style.use('ggplot')

In [ ]:
from simplekml import (Kml, OverlayXY, ScreenXY, Units, RotationXY,
                       AltitudeMode, Camera)


def make_kml(llcrnrlon, llcrnrlat, urcrnrlon, urcrnrlat,
             figs, colorbar=None, **kw):
    """TODO: LatLon bbox, list of figs, optional colorbar figure,
    and several simplekml kw..."""

    kml = Kml()
    altitude = kw.pop('altitude', 2e7)
    roll = kw.pop('roll', 0)
    tilt = kw.pop('tilt', 0)
    altitudemode = kw.pop('altitudemode', AltitudeMode.relativetoground)
    camera = Camera(latitude=np.mean([urcrnrlat, llcrnrlat]),
                    longitude=np.mean([urcrnrlon, llcrnrlon]),
                    altitude=altitude, roll=roll, tilt=tilt,
                    altitudemode=altitudemode)

    kml.document.camera = camera
    draworder = 0
    for fig in figs:  # NOTE: Overlays are limited to the same bbox.
        draworder += 1
        ground = kml.newgroundoverlay(name='GroundOverlay')
        ground.draworder = draworder
        ground.visibility = kw.pop('visibility', 1)
        ground.name = kw.pop('name', 'overlay')
        ground.color = kw.pop('color', '9effffff')
        ground.atomauthor = kw.pop('author', 'ocefpaf')
        ground.latlonbox.rotation = kw.pop('rotation', 0)
        ground.description = kw.pop('description', 'Matplotlib figure')
        ground.gxaltitudemode = kw.pop('gxaltitudemode',
                                       'clampToSeaFloor')
        ground.icon.href = fig
        ground.latlonbox.east = llcrnrlon
        ground.latlonbox.south = llcrnrlat
        ground.latlonbox.north = urcrnrlat
        ground.latlonbox.west = urcrnrlon

    if colorbar:  # Options for colorbar are hard-coded (to avoid a big mess).
        screen = kml.newscreenoverlay(name='ScreenOverlay')
        screen.icon.href = colorbar
        screen.overlayxy = OverlayXY(x=0, y=0,
                                     xunits=Units.fraction,
                                     yunits=Units.fraction)
        screen.screenxy = ScreenXY(x=0.015, y=0.075,
                                   xunits=Units.fraction,
                                   yunits=Units.fraction)
        screen.rotationXY = RotationXY(x=0.5, y=0.5,
                                       xunits=Units.fraction,
                                       yunits=Units.fraction)
        screen.size.x = 0
        screen.size.y = 0
        screen.size.xunits = Units.fraction
        screen.size.yunits = Units.fraction
        screen.visibility = 1

    kmzfile = kw.pop('kmzfile', 'overlay.kmz')
    kml.savekmz(kmzfile)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


def gearth_fig(llcrnrlon, llcrnrlat, urcrnrlon, urcrnrlat, pixels=1024):
    """Return a Matplotlib `fig` and `ax` handles for a Google-Earth Image."""
    aspect = np.cos(np.mean([llcrnrlat, urcrnrlat]) * np.pi/180.0)
    xsize = np.ptp([urcrnrlon, llcrnrlon]) * aspect
    ysize = np.ptp([urcrnrlat, llcrnrlat])
    aspect = ysize / xsize

    if aspect > 1.0:
        figsize = (10.0 / aspect, 10.0)
    else:
        figsize = (10.0, 10.0 * aspect)

    if False:
        plt.ioff()  # Make `True` to prevent the KML components from poping-up.
    fig = plt.figure(figsize=figsize,
                     frameon=False,
                     dpi=pixels//10)
    # KML friendly image.  If using basemap try: `fix_aspect=False`.
    ax = fig.add_axes([0, 0, 1, 1])
    ax.set_xlim(llcrnrlon, urcrnrlon)
    ax.set_ylim(llcrnrlat, urcrnrlat)
    return fig, ax

In [ ]:
import numpy.ma as ma
from netCDF4 import Dataset, date2index, num2date
from palettable import colorbrewer



In [ ]:
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
import requests
def listFD(url, ext=''):
    page = requests.get(url).text
    #print(page)
    soup = BeautifulSoup(page, 'html.parser')
    return [url + node.get('href') for node in soup.find_all('a') if (node.get('href').endswith(ext) and node.get('href').startswith('2'))]


In [ ]:
from os.path import exists

filename_viirs_proc = 'F:/data/cruise_data/saildrone/baja-2018/viirs_files.npy'
filename_viirs_sst = 'F:/data/cruise_data/saildrone/baja-2018/viirs_sst_data.npy'
filename_png = 'F:/data/cruise_data/saildrone/baja-2018/recent_viirs_sst.png'
filename_kmz = 'F:/data/cruise_data/saildrone/baja-2018/recent_viirs_sst.kmz'

if exists(filename_viirs_proc):
    print('reading:',filename_viirs_proc)
    proc_save = np.load(filename_viirs_proc)
else:
    proc_save = np.zeros((367,24,60))
  

inew_data=0 #initialize for new data
if exists(filename_viirs_sst):
    print('reading:',filename_viirs_sst)
    sst_new = np.load(filename_viirs_sst)
else:
    inew_data=1  
    
itotal_proc = 0

for incr_day in range(-1,1):
    d = datetime.today() + timedelta(days=incr_day)
    day_of_year = d.timetuple().tm_yday

    url = 'https://opendap.jpl.nasa.gov/opendap/OceanTemperature/ghrsst/data/GDS2/L3U/VIIRS_NPP/OSPO/v2.41/' \
            + str(d.year) + '/' + str(day_of_year).zfill(3) + '/'
    ext = 'nc'

    filenames=listFD(url, ext)
    ilen=len(filenames)
    for ic in range(1,ilen):
        file = filenames[ic]
        
        #save which files have been processed
        idyj=int(file[101:104])
        ihr=int(file[113:115])
        imin=int(file[115:117])
        if proc_save[idyj,ihr,imin]==1:
            continue #if file already processed skip
            
        proc_save[idyj,ihr,imin]=1
        itotal_proc = itotal_proc+1

        nc = Dataset(file)
        sst = nc.variables['sea_surface_temperature'][:,2500:3250,2500:3500]
        sst = np.flip(sst, 2)
        cnt = ma.count(sst)
        lat = nc.variables['lat'][2500:3250]
        lat = np.flip(lat, 0)
        lon = nc.variables['lon'][2500:3500]
        nc.close()
        if inew_data==1:
            sst_new = ma.copy(sst)
            inew_data = 0
        if cnt<5:
            continue
        print(ic,cnt,file)
        mask = sst_new < -10
        sst_new[mask] = sst[mask]
        print(lat[0],lat[-1])
        print(lon[0],lon[-1])
        
if itotal_proc>0:
    lat, lon = np.meshgrid(lat, lon)

    mask = (sst_new < -10)
    sst_new[mask]=-9999.
    sst_new2 = sst_new[0,:,:]
    sstx = ma.masked_values (sst_new2, -9999.)
    print(lon.shape,lat.shape,sstx.shape)
    sst3x = np.flip(sstx, 1)
    sst4x = np.flip(sst3x, 0)
    sst5x = ma.swapaxes(sst4x,0,1)
    print(lon.shape,lat.shape,sst5x.shape)

    pixels = 1024 * 10
    cmap = colorbrewer.get_map('Spectral', 'diverging', 11, reverse=True).mpl_colormap

    fig, ax = gearth_fig(llcrnrlon=lon.min(),
                                     llcrnrlat=lat.min(),
                                     urcrnrlon=lon.max(),
                                     urcrnrlat=lat.max(),
                                     pixels=pixels)
    cs = ax.pcolormesh(lon, lat, sst5x - 273.15, cmap=cmap, vmin = 10, vmax = 17)
    ax.set_axis_off()
    fig.savefig(filename_png, transparent=False, format='png')
    print('done')        
    print(lat[1,1],lat[1,-1],lon[1,1],lon[-1,1])
    make_kml(llcrnrlon=lon[1,1], llcrnrlat=lat[1,1],
             urcrnrlon=lon[-1,1], urcrnrlat=lat[1,-1],
             figs= [filename_png], colorbar='legend.png',
             kmzfile=filename_kmz, name='VIIRS Sea Surface Temperature')
    np.save(filename_viirs_proc, proc_save)
    sst_new.dump(filename_viirs_sst)

fig = plt.figure(figsize=(1.0, 4.0), facecolor=None, frameon=False)
ax = fig.add_axes([0.0, 0.05, 0.2, 0.9])
cb = fig.colorbar(cs, cax=ax)
cb.set_label('SST [C]', rotation=-90, color='k', labelpad=20)
fig.savefig('legend.png', transparent=False, format='png')  # Change transparent to True if your colorbar is not on space :)


In [ ]:
#now just do todays data
from os.path import exists

filename_png = 'F:/data/cruise_data/saildrone/baja-2018/today_viirs_sst.png'
filename_kmz = 'F:/data/cruise_data/saildrone/baja-2018/todat_viirs_sst.kmz'
inew_data=1

for incr_day in range(-1,1):
    d = datetime.today() + timedelta(days=incr_day)
    day_of_year = d.timetuple().tm_yday

    url = 'https://opendap.jpl.nasa.gov/opendap/OceanTemperature/ghrsst/data/GDS2/L3U/VIIRS_NPP/OSPO/v2.41/' \
            + str(d.year) + '/' + str(day_of_year).zfill(3) + '/'
    ext = 'nc'

    filenames=listFD(url, ext)
    ilen=len(filenames)
    for ic in range(1,ilen):
        file = filenames[ic]
        
        #save which files have been processed
        idyj=int(file[101:104])
        ihr=int(file[113:115])
        imin=int(file[115:117])
            
        nc = Dataset(file)
        sst = nc.variables['sea_surface_temperature'][:,2500:3250,2500:3500]
        sst = np.flip(sst, 2)
        cnt = ma.count(sst)
        lat = nc.variables['lat'][2500:3250]
        lat = np.flip(lat, 0)
        lon = nc.variables['lon'][2500:3500]
        nc.close()
        if inew_data==1:
            sst_new = ma.copy(sst)
            inew_data = 0
        if cnt<5:
            continue
        print(ic,cnt,file)
        mask = sst_new < -10
        sst_new[mask] = sst[mask]
        
    lat, lon = np.meshgrid(lat, lon)
    mask = (sst_new < -10)
    sst_new[mask]=-9999.
    sst_new2 = sst_new[0,:,:]
    sstx = ma.masked_values (sst_new2, -9999.)
    print(lon.shape,lat.shape,sstx.shape)
    sst3x = np.flip(sstx, 1)
    sst4x = np.flip(sst3x, 0)
    sst5x = ma.swapaxes(sst4x,0,1)
    print(lon.shape,lat.shape,sst5x.shape)

    pixels = 1024 * 10
    cmap = colorbrewer.get_map('Spectral', 'diverging', 11, reverse=True).mpl_colormap

    fig, ax = gearth_fig(llcrnrlon=lon.min(),
                                     llcrnrlat=lat.min(),
                                     urcrnrlon=lon.max(),
                                     urcrnrlat=lat.max(),
                                     pixels=pixels)
    cs = ax.pcolormesh(lon, lat, sst5x - 273.15, cmap=cmap, vmin = 10, vmax = 17)
    ax.set_axis_off()
    fig.savefig(filename_png, transparent=False, format='png')
    print('done')        
    print(lat[1,1],lat[1,-1],lon[1,1],lon[-1,1])
    make_kml(llcrnrlon=lon[1,1], llcrnrlat=lat[1,1],
             urcrnrlon=lon[-1,1], urcrnrlat=lat[1,-1],
             figs= [filename_png], colorbar='legend.png',
             kmzfile=filename_kmz, name='VIIRS Sea Surface Temperature')

fig = plt.figure(figsize=(1.0, 4.0), facecolor=None, frameon=False)
ax = fig.add_axes([0.0, 0.05, 0.2, 0.9])
cb = fig.colorbar(cs, cax=ax)
cb.set_label('SST [C]', rotation=-90, color='k', labelpad=20)
fig.savefig('legend.png', transparent=False, format='png')  # Change transparent to True if your colorbar is not on space :)


In [ ]:
pixels = 1024 * 10
cmap = colorbrewer.get_map('Spectral', 'diverging', 11, reverse=True).mpl_colormap
fig, ax = gearth_fig(llcrnrlon=lon.min(),
                                     llcrnrlat=lat.min(),
                                     urcrnrlon=lon.max(),
                                     urcrnrlat=lat.max(),
                                     pixels=pixels)
cs = ax.pcolormesh(lon, lat, sst5x - 273.15, cmap=cmap, vmin = 10, vmax = 17)
ax.set_axis_off()
fig.show()
 

In [ ]:
from copy import copy
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.mlab as mlab
print(sst5x.shape)
print(sst4x.shape)
print(sst3x.shape)
palette = copy(plt.cm.jet)
palette.set_over('r', 1.0)
palette.set_under('g', 1.0)
palette.set_bad(alpha = 0.0)
fig, (ax1, ax2) = plt.subplots(nrows=2, figsize=(6, 5.4))
# plot using 'continuous' color map
im = ax1.imshow(sst5x - 273.15, interpolation='bilinear',
#im = ax1.imshow(sst4x[540:560,280:289] - 273.15, interpolation='bilinear',
                cmap=palette,
                norm=colors.Normalize(vmin=10.0, vmax=17.0),
                aspect='auto',
                origin='lower')
                #extent=[x0, x1, y0, y1])
ax1.set_title('sst')

plt.show()
print(lon[283,546],lat[283,546])
